# Data Preparation

<img src="https://octodex.github.com/images/privateinvestocat.jpg" alt="Kit" title="Cat" width="350" height="200" />
*(image from octodex github)*


In [41]:
name= wg.Text(value='train')
display(name)

A Jupyter Widget

In [42]:
# Import Packages

#Dataframe packages
import zipfile
import json
import objectpath
import pandas as pd
import numpy as np
import os
from PIL import Image
import requests
from io import BytesIO
import cv2
import numpy as np

import ipywidgets as wg

## Training 

In [43]:
# Define Empty lists
d = None  
data = None  
score=[]
magnitude=[]
petid=[]

string = name.value + "_sentiment.zip"
# Read Zip File and Export a Dataset with the Score and the ID
filepath= os.path.join(r"C:\\Users\\alorenzodebrionne\\Documents\\Kaggle\\Pet.my\\", string)

with zipfile.ZipFile(filepath, "r") as z:
   for filename in z.namelist():  
      with z.open(filename) as f:  
         data = f.read()  
         d = json.loads(data.decode("utf-8"))
         json_tree = objectpath.Tree(d['documentSentiment'])
         result_tuple = tuple(json_tree.execute('$..score'))
         result_tuple2=tuple(json_tree.execute('$..magnitude'))
         score.append(result_tuple)
         magnitude.append(result_tuple2)
         
      petid.append(filename.replace('.json',''))

# Output with sentiment data for each pet
# Output with sentiment data for each pet
sentimental_analysis = pd.concat([ pd.DataFrame(petid, columns =['PetID']) ,pd.DataFrame(score, columns =['sentiment_document_score']),
                                                pd.DataFrame(magnitude, columns =['sentiment_document_magnitude'])],axis =1)

In [44]:
# Define Empty lists
d = None  
data = None  
description=[]
topicality=[]
imageid=[]

string=name.value + "_metadata.zip"
filepath= os.path.join(r"C:\\Users\\alorenzodebrionne\\Documents\\Kaggle\\Pet.my\\", string)
# Read Zip File and Export a Dataset with the Topicality and Description for each Image
with zipfile.ZipFile(filepath, "r") as z:
   for filename in z.namelist():  
       #Open the Zip File
      with z.open(filename) as f:  
          #Read the json File
          data = f.read()
          d = json.loads(data.decode("utf-8"))
          #Check if the file contains the parent label Annotations and export description and topicality
          if 'labelAnnotations' in d:
             json_tree = objectpath.Tree(d['labelAnnotations'])
             image_metadata1 = list(tuple(json_tree.execute('$..description')))
             image_metadata2 =  list(tuple(json_tree.execute('$..topicality')))

             #Create a list of all descriptions and topicality
             description.append(image_metadata1)
             topicality.append(image_metadata2)
         
             #Create a list with all image id name
             imageid.append(filename.replace('.json',''))


# Prepare the output by renaming all variables
description=pd.DataFrame(description)
topicality=pd.DataFrame(topicality)

new_names = [(i,'metadata_description_'+str(i)) for i in description.iloc[:, 0:].columns.values]
description.rename(columns = dict(new_names), inplace=True)

new_names = [(i,'metadata_topicality_'+str(i)) for i in topicality.iloc[:, 0:].columns.values]
topicality.rename(columns = dict(new_names), inplace=True)

# Output with sentiment data for each pet
image_metadata = pd.concat([ pd.DataFrame(imageid, columns =['ImageId']) ,topicality,description],axis =1)

# create the PetId variable
image_metadata['PetID'] = image_metadata['ImageId'].str.split('-').str[0]


##############
# TOPICALITY #
##############

image_metadata['metadata_topicality_mean'] = image_metadata.iloc[:,1:10].mean(axis=1)
image_metadata['metadata_topicality_mean']  = image_metadata.groupby(['PetID'])['metadata_topicality_mean'].transform('mean') 

image_metadata['metadata_topicality_max'] = image_metadata.iloc[:,1:10].max(axis=1)
image_metadata['metadata_topicality_max'] = image_metadata.groupby(['PetID'])['metadata_topicality_max'].transform(max)

image_metadata['metadata_topicality_min'] = image_metadata.iloc[:,1:10].min(axis=1)
image_metadata['metadata_topicality_min'] = image_metadata.groupby(['PetID'])['metadata_topicality_min'].transform(min)


image_metadata['metadata_topicality_0_mean']  = image_metadata.groupby(['PetID'])['metadata_topicality_0'].transform('mean')
image_metadata['metadata_topicality_0_max'] = image_metadata.groupby(['PetID'])['metadata_topicality_0'].transform(max)
image_metadata['metadata_topicality_0_min'] = image_metadata.groupby(['PetID'])['metadata_topicality_0'].transform(min)


###############
# DESCRIPTION #
###############

# Create Features from the Images
image_metadata['L_metadata_0_cat']=image_metadata['metadata_description_0'].str.contains("cat").astype(int)
image_metadata['L_metadata_0_dog'] =image_metadata['metadata_description_0'].str.contains("dog").astype(int)

image_metadata['L_metadata_any_cat']=image_metadata.apply(lambda row: row.astype(str).str.contains('cat').any(), axis=1)
image_metadata['L_metadata_any_dog']=image_metadata.apply(lambda row: row.astype(str).str.contains('dog').any(), axis=1)

image_metadata['L_metadata_0_cat_sum'] = image_metadata.groupby(image_metadata['PetID'])['L_metadata_0_cat'].transform('sum')
image_metadata['L_metadata_0_dog_sum'] = image_metadata.groupby(image_metadata['PetID'])['L_metadata_0_dog'].transform('sum')

image_metadata['L_metadata_any_cat_sum'] = image_metadata.groupby(image_metadata['PetID'])['L_metadata_any_cat'].transform('sum')
image_metadata['L_metadata_any_dog_sum'] = image_metadata.groupby(image_metadata['PetID'])['L_metadata_any_dog'].transform('sum')

image_metadata = image_metadata[['PetID','metadata_topicality_max','metadata_topicality_mean','metadata_topicality_min','metadata_topicality_0_mean','metadata_topicality_0_max','metadata_topicality_0_min','L_metadata_0_cat_sum','L_metadata_0_dog_sum','L_metadata_any_cat_sum','L_metadata_any_dog_sum']]
image_metadata=image_metadata.drop_duplicates('PetID')


In [45]:
d = None  
data = None  

color_score_mean=[]
color_score_min=[]
color_score_max=[]

color_pixelfrac_mean=[]
color_pixelfrac_min=[]
color_pixelfrac_max=[]

imageid=[]

# Read Zip File and Export a Dataset with the Score and the ID
with zipfile.ZipFile(filepath, "r") as z:
   for filename in z.namelist():  
      with z.open(filename) as f:  
          data = f.read()
          d = json.loads(data.decode("utf-8"))
          file_keys = list(d.keys())
          if  'imagePropertiesAnnotation' in file_keys:
              file_colors = d['imagePropertiesAnnotation']['dominantColors']['colors']
              file_crops = d['cropHintsAnnotation']['cropHints']
              
              file_color_score_mean = np.asarray([x['score'] for x in file_colors]).mean()
              file_color_pixelfrac_mean = np.asarray([x['pixelFraction'] for x in file_colors]).mean()
              
              file_color_score_min = np.asarray([x['score'] for x in file_colors]).min()
              file_color_pixelfrac_min = np.asarray([x['pixelFraction'] for x in file_colors]).min()

              
              file_color_score_max = np.asarray([x['score'] for x in file_colors]).max()
              file_color_pixelfrac_max = np.asarray([x['pixelFraction'] for x in file_colors]).max()
              
              
          #Create a list with all image id name
          imageid.append(filename.replace('.json',''))
          
          color_score_mean.append(file_color_score_mean)
          color_score_min.append(file_color_score_min)
          color_score_max.append(file_color_score_max)
          
          
          color_pixelfrac_mean.append(file_color_pixelfrac_mean)
          color_pixelfrac_min.append(file_color_pixelfrac_min)
          color_pixelfrac_max.append(file_color_pixelfrac_max)

image_properties = pd.concat([pd.DataFrame({'ImageId':imageid}),pd.DataFrame({'metadata_color_pixelfrac_mean':color_pixelfrac_mean}), pd.DataFrame({'metadata_color_pixelfrac_min':color_pixelfrac_min}),pd.DataFrame({'metadata_color_pixelfrac_max':color_pixelfrac_max}),pd.DataFrame({'metadata_color_score_mean':color_score_mean}),pd.DataFrame({'metadata_color_score_min':color_score_min}),pd.DataFrame({'metadata_color_score_max':color_score_max})],axis=1)

# create the PetId variable
image_properties['PetID'] = image_properties['ImageId'].str.split('-').str[0]


##############
# COLOR INFO #
##############
image_properties['metadata_color_pixelfrac_mean']  = image_properties.groupby(['PetID'])['metadata_color_pixelfrac_mean'].transform('mean') 
image_properties['metadata_color_pixelfrac_min']  = image_properties.groupby(['PetID'])['metadata_color_pixelfrac_min'].transform(min) 
image_properties['metadata_color_pixelfrac_max']  = image_properties.groupby(['PetID'])['metadata_color_pixelfrac_max'].transform(max) 

image_properties['metadata_color_score_mean']  = image_properties.groupby(['PetID'])['metadata_color_score_mean'].transform('mean') 
image_properties['metadata_color_score_min']  = image_properties.groupby(['PetID'])['metadata_color_score_min'].transform(min) 
image_properties['metadata_color_score_max']  = image_properties.groupby(['PetID'])['metadata_color_score_max'].transform(max) 

image_properties=image_properties.drop_duplicates('PetID')

In [ ]:
def variance_of_laplacian(data):
    image = cv2.imdecode(np.frombuffer(data, np.uint8), 1)      
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)   
    # compute the Laplacian of the image and then return the focus
    # measure, which is simply the variance of the Laplacian
    return cv2.Laplacian(gray, cv2.CV_64F).var() 

blur=[]
image_pixel=[]
imageid =[]

string=name.value + "_images.zip"
filepath= os.path.join(r"C:\\Users\\alorenzodebrionne\\Documents\\Kaggle\\Pet.my\\", string)

#Read the Zip File    
with zipfile.ZipFile(filepath,"r") as zfile:
      for filename in zfile.namelist():
              
              #Blur 
              data = zfile.read(filename)
              # Pixels
              with Image.open( BytesIO(data)) as pixel:
                  width, height = pixel.size
              
              pixel = width*height
              
              #image pixel size for each image
              image_pixel.append(pixel)
              #blur for each image
              blur.append(variance_of_laplacian(data))
              #image id
              imageid.append(filename.replace('.jpg',''))
          
          
# Join Pixel, Blur and Image ID
image_quality = pd.concat([ pd.DataFrame(imageid, columns =['ImageId']) ,pd.DataFrame(blur, columns =['blur']),
                                        pd.DataFrame(image_pixel,columns=['pixel'])],axis =1)

# create the PetId variable
image_quality['PetID'] = image_quality['ImageId'].str.split('-').str[0]

#Mean of the Mean
image_quality['pixel_mean'] = image_quality.groupby(['PetID'])['pixel'].transform('mean')
image_quality['blur_mean'] = image_quality.groupby(['PetID'])['blur'].transform('mean') 

image_quality['pixel_min'] = image_quality.groupby(['PetID'])['pixel'].transform('min') 
image_quality['blur_min'] = image_quality.groupby(['PetID'])['blur'].transform('min')

image_quality['pixel_max'] = image_quality.groupby(['PetID'])['pixel'].transform('max') 
image_quality['blur_max'] = image_quality.groupby(['PetID'])['blur'].transform('max')

image_quality['pixel_sum'] = image_quality.groupby(['PetID'])['pixel'].transform('sum')
image_quality['blur_sum'] = image_quality.groupby(['PetID'])['blur'].transform('sum')


image_quality = image_quality.drop(['blur','pixel'], 1)
image_quality=image_quality.drop_duplicates('PetID')

In [ ]:
#Load Data

if name.value == "train":
    df =pd.read_csv('https://raw.githubusercontent.com/alexlorenzo/Pet.my/master/train.csv')
    
    
if name.value == "test":
    df =pd.read_csv('https://raw.githubusercontent.com/alexlorenzo/Pet.my/master/test.csv')

#test = pd.read_csv('https://raw.githubusercontent.com/alexlorenzo/Pet.my/master/test.csv')
breed =pd.read_csv('https://raw.githubusercontent.com/alexlorenzo/Pet.my/master/breed_labels.csv',usecols=["BreedID", "BreedName"]) #A pet could have multiple breed
color =pd.read_csv('https://raw.githubusercontent.com/alexlorenzo/Pet.my/master/color_labels.csv') #A pet could have multiple colors
state =pd.read_csv('https://raw.githubusercontent.com/alexlorenzo/Pet.my/master/state_labels.csv')

# Add information about color, breed, state and sentiment data
df = (pd.merge(df, breed.rename(columns={"BreedName": "BreedName1"}),  how='left', left_on=['Breed1'], right_on = ['BreedID']).drop('BreedID', axis=1))
df = (pd.merge(df, breed.rename(columns={"BreedName": "BreedName2"}),  how='left', left_on=['Breed2'], right_on = ['BreedID']).drop('BreedID', axis=1))

df = (pd.merge(df, color.rename(columns={"ColorName": "ColorName1"}),  how='left', left_on=['Color1'], right_on = ['ColorID']).drop('ColorID', axis=1))
df = (pd.merge(df, color.rename(columns={"ColorName": "ColorName2"}),  how='left', left_on=['Color2'], right_on = ['ColorID']).drop('ColorID', axis=1))
df = (pd.merge(df, color.rename(columns={"ColorName": "ColorName3"}),  how='left', left_on=['Color3'], right_on = ['ColorID']).drop('ColorID', axis=1))
df = (pd.merge(df, state,  how='left', left_on=['State'], right_on = ['StateID']).drop('StateID', axis=1))

df = (pd.merge(df, sentimental_analysis,  how='left', left_on=['PetID'], right_on = ['PetID']))

# Add information about Metadata Images
df = (pd.merge(df, image_metadata,  how='left', left_on=['PetID'], right_on = ['PetID']))
# Add information about Metadata Images
df = (pd.merge(df, image_properties,  how='left', left_on=['PetID'], right_on = ['PetID']))


# Add information about quality Images
df= (pd.merge(df, image_quality,  how='left', left_on=['PetID'], right_on = ['PetID']))


### Features Engineering

In [ ]:
# Color (Create a Flag pet has 1 color, 2 colors, 3 colors)
df['L_Color1'] = (pd.isnull(df['ColorName3']) & pd.isnull(df['ColorName2']) & pd.notnull(df['ColorName1'])).astype(int)
df['L_Color2'] = (pd.isnull(df['ColorName3']) & pd.notnull(df['ColorName2']) & pd.notnull(df['ColorName1'])).astype(int)
df['L_Color3'] = (pd.notnull(df['ColorName3']) & pd.notnull(df['ColorName2']) & pd.notnull(df['ColorName1'])).astype(int)

# Breed (create a flag if the pet has 1 breed or 2)
df['L_Breed1'] = (pd.isnull(df['BreedName2']) & pd.notnull(df['BreedName1'])).astype(int)
df['L_Breed2'] = (pd.notnull(df['BreedName2']) & pd.notnull(df['BreedName1'])).astype(int)

#Name (create a flag if the name is missing, with less than two letters)
df['L_Name_missing'] =  (pd.isnull(df['Name'])).astype(int)
df['Name_Length']=df['Name'].str.len() 

#Description 
df['Description_Length']=df['Description'].str.len() 

# Fee Amount
df['L_Fee_Free'] =  (df['Fee']==0).astype(int)

#Add the Number of Pets per Rescuer 
pets_total = df.groupby(['RescuerID']).size().reset_index(name='N_pets_total')
df= pd.merge(df, pets_total, left_on='RescuerID', right_on='RescuerID', how='inner')
df.count()

# No photo
df['L_NoPhoto'] =  (df['PhotoAmt']==0).astype(int)

#No Video
df['L_NoVideo'] =  (df['VideoAmt']==0).astype(int)

#Log Age 
df['Log_Age']= np.log(df.Age+1) 

#Negative Score 
df['L_scoreneg'] =  (df['sentiment_document_score']<0).astype(int)

#Quantity Amount >5
df.loc[df['Quantity'] > 5, 'Quantity'] = 5

In [ ]:
# Cannot be used for this analysis (IDs, Texts...)
df = df.drop(["Name","Description","BreedName2","ColorName3",'Name','Breed1','Breed2','RescuerID','Description',
                            'BreedName1','Color1', 'Color2', 'Color3','Age','State'],axis=1)

for col in ['sentiment_document_score', 'sentiment_document_magnitude','metadata_topicality_max','metadata_topicality_mean','metadata_topicality_min',
           'metadata_topicality_0_mean','metadata_topicality_0_max','metadata_topicality_0_min','L_metadata_0_dog_sum',
           'L_metadata_0_cat_sum','L_metadata_any_dog_sum','L_metadata_any_cat_sum','pixel_mean','pixel_min','pixel_max','pixel_sum',
           'blur_min','blur_max','blur_sum','blur_mean']:
    df[col].fillna((df[col].median()), inplace=True)
    
# replacing na values with No Color 
df["ColorName2"].fillna("No Color", inplace = True) 


In [ ]:
print(df)

In [ ]:
#One Hot Encoding
df = pd.concat([df.drop('StateName', axis=1),pd.get_dummies(df['StateName'], prefix='State')], axis=1)

col=['ColorName1','ColorName2','Health', 'Gender', 'Dewormed','Type','MaturitySize', 'Sterilized','Vaccinated']
for i in col:
    df = pd.concat([df.drop(i, axis=1),pd.get_dummies(df[i], prefix=i)], axis=1)

In [ ]:
output= name.value + "features.csv"
filepath= os.path.join("C:\Users\alorenzo\Documents\Kaggle\Pet.my\",output)
df.to_csv(filepath)